# Elastic Stackの構築手順

## 基本概念

実際の構築の前に、クラスタに関する基本概念を説明します。

### Elasticsearchクラスタ

Elasticsearchは複数台のサーバで起動しているElasticsearch間で連携して、分散処理を行うことができます。  
以下のような要素によって構成されます。

![images/01_basic_concept.png](images/01_basic_concept.png)

**※Node数が3、Shard数が3、Replica数が1の場合の例**

|名称|意味|
|----|----|
|ノード<br>Node|動作しているElasticsearch Serverのことです。|
|クラスタ<br>Cluster|Nodeの集合です。|
|インデックス<br>Index|リレーショナルデータベースでのDatabaseに相当する概念です。<br>Indexは複数のType(テーブルに相当)を持ち、<br>Typeは複数のDocument(レコードに相当)を持ちます。<br>1つのClusterに複数のIndexを作成できます。|
|シャード<br>Shard|Indexを分割した断片です。Shardの単位でNodeに配置されます。|
|プライマリシャード<br>Primary Shard|読み書きの対象となるShardです。|
|レプリカシャード<br>Replica Shard|読み取り専用の、Primary Shardの複製です。|


複数のNodeを生成してそれぞれを物理マシン等に分散配置することで、蓄積するデータ量や検索の速度を向上できますが、
用意した各物理マシンのスペックを十分に発揮させてElasticsearchのクラウドを稼働させるには、蓄積しているデータも分割して分散配置する必要があります。  
Shardはそのための分割の単位です。分割するShard数を設定することで、自動的にCluster内のNodeにPrimary Shardとして配置されます。  
さらに設定されたReplica数だけ、各Primary Shardが複製され、可用性を確保します。

## ノードの役割の違い

### Nodeの種類と使い分け

Nodeの役割はデータの蓄積だけではありません。Clusterの管理やデータの加工などの役割を持った、複数のNodeの種類が存在します。  
種類ごとの役割は次の通りです。

![images/01_node_type.png](images/01_node_type.png)

| 種別              | 役割                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | elasticsearch.ymlの設定内容 |
|-------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------| |
| Coordinating Node | 検索リクエストやインデクシングリクエストなどを受けつけるノードです。<br>このノードは、リクエストを受けた際に必要なデータを持っているノードに対してリクエストを転送し、返ってきた結果をマージしてからレスポンスとして返す役割を持ちます。|**※特になし**<br><br>全てのノードは、設定によらずCoordinating Nodeとして動作します。実際のどのノードをCoordinating Nodeとするかは利用方法次第となります。 |
| Master Node       | クラスタの管理を行うノードです。このノードはクラスタに属するノードを把握したり、データを保持するシャードを決定するなど、クラスタ全体に関わる役割を持ちます。<br>右の設定をしたノードはmaster-eligible nodeと呼ばれるMasterの候補となります。<br>それらの候補が連携し、うち1台がMaster Nodeとして選出されるよう動きます。 |**node.master=true**<br>(デフォルトはtrue)<br><br>つまりデフォルトでmaster-eligible nodeとなります。<br>なお、特にdata nodeの役割を持たずにmaster-eligible nodeの役割のみを持つノードをdedicated master nodeと呼びます。 |
| Data Node         | Elasticsearchのデータを保持するノードです。このノードはデータを保持し、クエリに対応した結果を返す役割を持ちます。<br><br>Data NodeはMaster Nodeとは兼任することも、別にすることもできます。<br>特にノード数の少ないクラスタ構成（～5台程度）では、すべてのノードをMaster NodeかつData Nodeとし、ノード数が増えた場合には役割を分けると良いでしょう。 |**node.data=true**<br>(デフォルトはtrue)<br><br>つまりデフォルトでData Nodeとなります。 |
| Client Node       | Master Node、Data Nodeのいずれの役割も担わず、Coordinating Nodeの役割に特化したノードの便宜的な呼び方です。<br><br>Kibanaを配置したサーバにElasticsearchのClient nodeを配置することで、他のElasticsearchのクラスタに負荷を掛けないようにしたり、Elasticsearchのロードバランサのように振る舞わせるようにしたりなどの用途で用います。|**node.master=false**<br>**node.data=false** |
| Ingest Node       | 実際のインデクシング処理を行う前に、ドキュメントを加工することができるノードです。<br>いくつかの加工処理を記述した「pipeline」を登録しておけば、その内容に従った処理をすることが可能です。<br>また、pipelineの振る舞いを検査するためのSimulate Pipeline APIも提供されています。 | **node.ingest=true**<br>(デフォルトはtrue)<br><br>つまりデフォルトでIngest Nodeとなります。 |

## クラスタの構築

### 構成と手順の概要

ここからは、実際のElasticsearchクラスタを構築します。  

このNotebookで構成するのは、以下の図に示すような構成（Data Node3台、Master Node1台、Client Node1台)です。  
もし複数台のサーバがなかったり、まずElastic Stackを1台のサーバで試したいだけであったりすれば、もう1つのNotebook「[01_Setup_1server](01_Setup_1server.ipynb)」での構築を試してください。

![images/01_5server.png](images/01_5server.png)

この後のNotebookでの大まかな手順は次の通りです。  

1. jdkのインストール
2. Elasticsearchのインストール
3. elasticsearch.ymlの設定
4. Elasticsearchの起動
5. Logstashのインストール
6. Logstashの設定
7. サンプルデータの投入

なおLogstashとは、Elastic Stackをを構成するプロダクトの１つです。  
様々な形式のデータの読み込み、加工、格納を処理するパイプラインを簡単に構築できます。  
  
詳細は[04_Store_Data](04_Store_Data.ipynb)で説明しますが、ここではサンプルデータを投入したいので、先にインストールまでしておきます。

### 準備

このNotebookで環境を構築したい先のサーバを、次のセルで設定してください。

In [98]:
%env ES_CLIENT=XXX.XXX.XXX.232:9200
%env ES_MASTER=XXX.XXX.XXX.233:9200
%env ES_DATA1=XXX.XXX.XXX.234:9200
%env ES_DATA2=XXX.XXX.XXX.235:9200
%env ES_DATA3=XXX.XXX.XXX.236:9200
%env LOGSTASH_HOST=XXX.XXX.XXX.232
%env INDEX=meteorological-data-*
%env TYPE=logs

env: ES_CLIENT=XXX.XXX.XXX.232:9200
env: ES_MASTER=XXX.XXX.XXX.233:9200
env: ES_DATA1=XXX.XXX.XXX.234:9200
env: ES_DATA2=XXX.XXX.XXX.235:9200
env: ES_DATA3=XXX.XXX.XXX.236:9200
env: LOGSTASH_HOST=XXX.XXX.XXX.232
env: INDEX=meteorological-data-*
env: TYPE=logs


各サーバにansibleコマンドを発行するため、
ansibleを実行可能なユーザアカウントとそのSSHキーを次のセルで設定してください。

In [104]:
#elasticsearchをインストールするサーバーでansibleコマンドを実行するユーザー
USER='ansible'

#公開鍵認証を行う場合の秘密鍵のパス
KEYPATH='~/.ssh/ansible_id_rsa'

ansibleのインベントリファイルを作成します。
次のセルを実行して得られる出力内容を/etc/ansible/hostsとして保存します。

[client-nodes]  
< ES_CLIENT >  
[master-nodes]  
< ES_MASTER >  
[data-nodes]  
< ES_DATA1 >  
< ES_DATA2 >  
< ES_DATA3 >  
[logstash-server]  
< LOGSTASH_HOST >

ここまでの設定が問題ないか確認します。  
次のセルを実行して、ansibleコマンドが各サーバで実行できることを確認してください。

In [37]:
!ansible all -m ping -i /etc/ansible/hosts -u $USER --private-key=$KEYPATH

XXX.XXX.XXX.234 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
XXX.XXX.XXX.233 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
XXX.XXX.XXX.232 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
XXX.XXX.XXX.235 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
XXX.XXX.XXX.236 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


各サーバから、次のようにSUCCESSという応答が返ってくれば問題なく実行できています。

この後から実際の構築作業を開始します。

### jdkのインストール

ElasticsearchはJavaで実装されており、実行するためにはJavaの環境が必要です。  
Elasticsearch-5.0.0ではバージョン1.8.0_73以降のものが推奨されています。

[Oracleの公式サイト](http://www.oracle.com/technetwork/java/javase/downloads/java-archive-javase8-2177648.html#jdk-8u112-oth-JPR)からjdk-8u112のrpm形式（jdk-8u112-linux-x64.rpm）をダウンロードしてください。  
ダウンロード先のディレクトリは**/tmp**とします。

次のコマンドを実行して、ダウンロードしたjdkをインストールします。

In [62]:
!ansible-playbook -i /etc/ansible/hosts playbooks/install_jdk.yml -u $USER --private-key=$KEYPATH


PLAY [all] *********************************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.234]
ok: [XXX.XXX.XXX.233]
ok: [XXX.XXX.XXX.232]
ok: [XXX.XXX.XXX.236]
ok: [XXX.XXX.XXX.235]

TASK [download jdk-8u112-linux-x64.rpm] ****************************************
ok: [XXX.XXX.XXX.233]
ok: [XXX.XXX.XXX.234]
ok: [XXX.XXX.XXX.232]
ok: [XXX.XXX.XXX.235]
ok: [XXX.XXX.XXX.236]

TASK [install jdk] *************************************************************
ok: [XXX.XXX.XXX.234]
ok: [XXX.XXX.XXX.233]
ok: [XXX.XXX.XXX.232]
ok: [XXX.XXX.XXX.236]
ok: [XXX.XXX.XXX.235]

TASK [set JAVA_HOME] ***********************************************************
ok: [XXX.XXX.XXX.233]
ok: [XXX.XXX.XXX.234]
ok: [XXX.XXX.XXX.232]
ok: [XXX.XXX.XXX.235]
ok: [XXX.XXX.XXX.236]

TASK [set PATH] ****************************************************************
ok: [XXX.XXX.XXX.234]
ok: [XXX.XXX.XXX.232]
ok: [XXX.XXX.XXX.233]
ok

### Elasticsearchのインストール

次のコマンドを実行して、Elasticsearchをインストールします。

In [63]:
!ansible-playbook playbooks/install_elasticsearch.yml -i /etc/ansible/hosts -u $USER --private-key=$KEYPATH


PLAY [all] *********************************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.233]
ok: [XXX.XXX.XXX.234]
ok: [XXX.XXX.XXX.232]
ok: [XXX.XXX.XXX.236]
ok: [XXX.XXX.XXX.235]

TASK [install elasticsearch 5.0.0] *********************************************
ok: [XXX.XXX.XXX.232]
ok: [XXX.XXX.XXX.234]
ok: [XXX.XXX.XXX.233]
ok: [XXX.XXX.XXX.236]
ok: [XXX.XXX.XXX.235]

PLAY RECAP *********************************************************************
XXX.XXX.XXX.232              : ok=2    changed=0    unreachable=0    failed=0   
XXX.XXX.XXX.233              : ok=2    changed=0    unreachable=0    failed=0   
XXX.XXX.XXX.234              : ok=2    changed=0    unreachable=0    failed=0   
XXX.XXX.XXX.235              : ok=2    changed=0    unreachable=0    failed=0   
XXX.XXX.XXX.236              : ok=2    changed=0    unreachable=0    failed=0   



### elasticsearch.ymlの設定

elasticsearch.ymlの設定を変更します。

elasticsearch.ymlの設定内容は各nodeごとに異なります。  
次の表に、nodeの種類別に設定を記載します。
デフォルトファイルの変更箇所を示すので、環境に合わせてelasticsearchのサーバ上にあるファイルを適宜変更してください。  
パスは/etc/elasticsearch/elasticsearch.ymlです。

nodeの種類と役割については[ノードの役割の違い](#ノードの役割の違い)で説明しています。

例）Master Nodeはnode.dataの値がfalseなので、elasticsearch.ymlに 「**node.data:false**」 と記述します

|                                  | Client Node    | Master Node    | Data Node      |
|----------------------------------|----------------|----------------|----------------|
| cluster.name<br>クラスタの名前です。任意の名前で構いませんがすべてのノードで同一の名前を設定してください。                      | cluster-name   | cluster-name   | cluster-name   |
| network.host<br>他のノードからアクセスする際に指定するホスト名またはIPアドレスを示します。<br>ノード自身のホスト名またはIPアドレスで書きかえてください。| 自身のホストアドレス<br>(例)192.0.0.1        | 自身のホストアドレス        | 自身のホストアドレス        |
| node.master<br>  Master Nodeになるかどうかを決める設定です。<br>この項目はデフォルトではelasticsearch.ymlに記述がないので追記します。                      | false          | true           | false          |
| node.data<br>Data Nodeになるかどうかを決める設定です。<br>この項目はデフォルトではelasticsearch.ymlに記述がないので追記します。                        | false          | false          | true           |
| discovery.zen.ping.unicast.hosts<br>クラスタに含まれるmaster-eligible nodeのホスト名またはIPアドレスを指定します。  | Master Nodeのホストアドレス<br>(例)[192.0.0.1] | Master Nodeのホストアドレス | Master Nodeのホストアドレス|

### Elasticsearchの起動

Elasticsearchを起動します。

In [64]:
!ansible-playbook playbooks/start_cluster.yml -i /etc/ansible/hosts -u $USER --private-key=$KEYPATH


PLAY [all] *********************************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.233]
ok: [XXX.XXX.XXX.234]
ok: [XXX.XXX.XXX.232]
ok: [XXX.XXX.XXX.235]
ok: [XXX.XXX.XXX.236]

TASK [start elasticsearch] *****************************************************
ok: [XXX.XXX.XXX.233]
ok: [XXX.XXX.XXX.232]
ok: [XXX.XXX.XXX.234]
ok: [XXX.XXX.XXX.235]
ok: [XXX.XXX.XXX.236]

PLAY RECAP *********************************************************************
XXX.XXX.XXX.232              : ok=2    changed=0    unreachable=0    failed=0   
XXX.XXX.XXX.233              : ok=2    changed=0    unreachable=0    failed=0   
XXX.XXX.XXX.234              : ok=2    changed=0    unreachable=0    failed=0   
XXX.XXX.XXX.235              : ok=2    changed=0    unreachable=0    failed=0   
XXX.XXX.XXX.236              : ok=2    changed=0    unreachable=0    failed=0   



各サーバの起動は次のコマンドで確認してください。

In [65]:
!ansible all -m shell -a "sudo systemctl status elasticsearch" -u $USER --private-key=$KEYPATH

XXX.XXX.XXX.233 | SUCCESS | rc=0 >>
● elasticsearch.service - Elasticsearch
   Loaded: loaded (/usr/lib/systemd/system/elasticsearch.service; disabled; vendor preset: disabled)
   Active: active (running) since Tue 2016-12-13 13:21:45 JST; 3 days ago
     Docs: http://www.elastic.co
  Process: 23237 ExecStartPre=/usr/share/elasticsearch/bin/elasticsearch-systemd-pre-exec (code=exited, status=0/SUCCESS)
 Main PID: 23240 (java)
   CGroup: /system.slice/elasticsearch.service
           └─23240 /bin/java -Xms2g -Xmx2g -XX:+UseConcMarkSweepGC -XX:CMSInitiatingOccupancyFraction=75 -XX:+UseCMSInitiatingOccupancyOnly -XX:+DisableExplicitGC -XX:+AlwaysPreTouch -server -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dlog4j.shutdownHookEnabled=false -Dlog4j2.disable.jmx=true -Dlog4j.skipJansi=true -XX:+HeapDumpOnOutOfMemoryError -Des.path.home=/usr/share/elasticsearch -cp /usr/share/elasticsearch/lib/elasticsearch-5.0.

次のように  
**Active: active (running)**    
と表示されれば正常に起動しています。

### Logstashのインストール

次のコマンドを実行して、Logstashをインストールします。

In [67]:
!ansible-playbook -i /etc/ansible/hosts playbooks/install_logstash.yml -u $USER --private-key=$KEYPATH


PLAY [logstash-server] *********************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.232]

TASK [install logstash] ********************************************************
ok: [XXX.XXX.XXX.232]

PLAY RECAP *********************************************************************
XXX.XXX.XXX.232              : ok=2    changed=0    unreachable=0    failed=0   



## クラスタのステータスの確認方法と監視方法

クラスタのステータスを確認するためにはCluster APIを用います。  
概要を表示したい場合にはCluster Health APIを用います。  
クラスタの状態や、クラスタに属しているノードの数、シャードの数などを見ることができます。

In [106]:
!curl -XGET http://$ES_CLIENT/_cluster/health?pretty

{
  "cluster_name" : "cluster-name",
  "status" : "green",
  "timed_out" : false,
  "number_of_nodes" : 5,
  "number_of_data_nodes" : 3,
  "active_primary_shards" : 85,
  "active_shards" : 170,
  "relocating_shards" : 0,
  "initializing_shards" : 0,
  "unassigned_shards" : 0,
  "delayed_unassigned_shards" : 0,
  "number_of_pending_tasks" : 0,
  "number_of_in_flight_fetch" : 0,
  "task_max_waiting_in_queue_millis" : 0,
  "active_shards_percent_as_number" : 100.0
}


上記のコマンドを実行して得られる出力の中で、**status**や**number_of_nodes**を見れば  
クラスタの状態およびクラスタ内のノード数を確認することができます。  
データ投入直後などは、クラスタの状態がyellowになっている場合があります。その際はgreenになったことを確認してから検索などを行ってください。

出力例

より詳細な情報を確認したい場合にはCluster Stats APIを用います。

In [69]:
!curl -XGET http://$ES_CLIENT/_cluster/stats?pretty

{
  "_nodes" : {
    "total" : 5,
    "successful" : 5,
    "failed" : 0
  },
  "cluster_name" : "cluster-name",
  "timestamp" : 1481881240987,
  "status" : "green",
  "indices" : {
    "count" : 17,
    "shards" : {
      "total" : 170,
      "primaries" : 85,
      "replication" : 1.0,
      "index" : {
        "shards" : {
          "min" : 10,
          "max" : 10,
          "avg" : 10.0
        },
        "primaries" : {
          "min" : 5,
          "max" : 5,
          "avg" : 5.0
        },
        "replication" : {
          "min" : 1.0,
          "max" : 1.0,
          "avg" : 1.0
        }
      }
    },
    "docs" : {
      "count" : 470,
      "deleted" : 0
    },
    "store" : {
      "size_in_bytes" : 2595256,
      "throttle_time_in_millis" : 0
    },
    "fielddata" : {
      "memory_size_in_bytes" : 0,
      "evictions" : 0
    },
    "query_cache" : {
      "memory_size_in_bytes" : 0,
      "total_count" : 0,
      "hit_count" : 0,
      "miss_count" : 0,
      "cac

このコマンドでは、シャードやメモリに関する情報を取得することができます。  
詳細はElasticsearch Referenceの[Cluster Stats](https://www.elastic.co/guide/en/elasticsearch/reference/current/cluster-stats.html)を参照してください。

### Monitoringによる監視

Monitoring([公式ページ](https://www.elastic.co/guide/en/x-pack/current/xpack-monitoring.html))はサブスクリプションを購入すると利用できる「X-Pack」をインストールすることで有効になる機能です。  

Elasticsearchの稼働状況を監視するプラグインで、CPU使用率やディスクサイズ、Latencyなどの情報を取得することができます。  
取得した情報はElasticsearchに登録されるので、あとから遡ってクラスタのステータスを確認することも可能です。  

また、X-Packに含まれるWatcherを用いることで、特定の条件下で通知を行うこともできます。  
例） CPU使用率が特定の値を超えた、ディスク空き容量が一定量を下回ったなど。

## Logstashの設定

サンプルデータをElasticsearchに登録するためのlogstash設定ファイルは[logstash_conf](logstash_conf)に配置してあるmain.confを用います。  
main.confの中で、Elasticsearchのホスト名を指定する箇所があるので、[準備](#準備)で定義した  

ES_CLIENT: < ES_CLIENT >  

で書き換えてください。  
具体的な修正箇所は次の通りです


ここで利用するmain.confは標準入力で読み込んだCSV形式の内容をElasticsearchに格納するための設定です。  
具体的な意味は[04_Store_Data](04_Store_Data.ipynb)で説明します。

Logstashをインストールしたサーバーの ~/ 配下に[logstash_conf](logstash_conf)のmain.confをコピーしてください。

### データ型の定義（マッピング定義）

サンプルデータの登録をする前に、登録するデータの「データ型」を定義しておく必要があります。  
データ型の種類およびマッピング定義の詳細に関しては[05_Indexing](05_Indexing.ipynb)を参照してください。

次のコマンドを実行してマッピング定義を行います。  

In [113]:
%%bash
curl -XPUT "http://$ES_CLIENT/_template/weather" -d @- << EOF
{
  "template" : "meteorological-data-*",
  "mappings": {
    "logs": {
      "dynamic_templates" : [
        {
          "my_strings" : {
            "match_mapping_type" : "string",
            "mapping" : {
              "type" : "keyword"
            }
          }
        }
      ],
      "properties": {
        "atmospheric_pressure": {
          "type": "float"
        },
        "sea_level_pressure":{
          "type": "float"
        },
        "precipitation_day": {
          "type": "integer"
        },
        "precipitation_day": {
          "type": "float"
        },
        "precipitation_max_hour":{
          "type":"float"
        },
        "precipitation_max_10min":{
          "type":"float"
        },
        "temperature_avg":{
          "type":"float"
        },
        "temperature_max":{
          "type":"float"
        },
        "temperature_min":{
          "type":"float"
        },
        "humidity_avg":{
          "type":"float"
        },
        "humidity_min":{
          "type":"float"
        },
        "wind_speed_avg":{
          "type":"float"
        },
        "wind_speed_max":{
          "type":"float"
        },
        "wind_speed_max_moment":{
          "type":"float"
        },
        "sunshine_duration":{
          "type":"float"
        },
        "snowfall":{
          "type":"float"
        },
        "snowfall_max":{
          "type":"float"
        }
      }
    }
  }
}
EOF

{"acknowledged":true}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1394  100    21  100  1373    687  44930 --:--:-- --:--:-- --:--:-- 45766


### サンプルデータの投入

以降の章で扱うサンプルデータを登録します。  
ここでは気象庁から提供されている気象データを利用します。
[気象庁「過去の気象データ・ダウンロード」](http://www.data.jma.go.jp/gmd/risk/obsdl/)

公開されているデータのうち、2015年の東京のデータ(1年分)を本Notebookで利用するCSV形式に変換し、サンプルデータとして利用します([tokyo2015.csv](sample_data/tokyo2015.csv))。

サンプルデータと気象庁データの項目は、次のように対応しています。

| サンプルデータの項目 | 気象庁データの項目 | 備考 |
|-----------|--------------------|------|
| @timestamp | 年月日 ||
| location | 地点 | 気象庁データに直接の記載はないため、変換時に追加した |
| atmospheric_pressure | 日平均現地気圧 ||
| sea_level_pressure | 日平均海面気圧 ||
| precipitation_day | 降水量の日合計 ||
| precipitation_max_hour | 1時間降水量の日最大 ||
| precipitation_max_10min | 10分間降水量の日最大 ||
| temperature_avg | 日平均気温 ||
| temperature_max | 日最高気温 ||
| temperature_min | 日最低気温 ||
| humidity_avg | 日平均相対湿度 ||
| humidity_min | 日最小相対湿度 ||
| wind_speed_avg | 日平均風速 ||
| wind_speed_max | 日最大風速 ||
| wind_direction | 日最大風速（風向） ||
| wind_speed_max_moment | 日最大瞬間風速 ||
| wind_direction_max_moment | 日最大瞬間風速（風向） ||
| sunshine_duration | 日照時間 ||
| snowfall | 降雪量の日合計 ||
| snowfall_max | 日最深積雪 ||
| information_daytime | 天気概況（昼：06時～18時） ||
| information_night | 天気概況（夜：18時～翌日06時） |||

気象庁データの利用あたっては気象庁が公開している[利用規約](http://www.jma.go.jp/jma/kishou/info/coment.html)を参照してください。

登録するデータの内容は次のコマンドで確認できます。1行目がデータの項目を表し、2行目以降が実際のデータです。  
Elasticsearchは慣習として時刻情報を@timestampというフィールドに格納します。

In [71]:
!head -n 5 sample_data/tokyo2015.csv

Logstashのサーバーにサンプルデータをコピーします。   
Logstashをインストールしたサーバーの ~/ 配下に[sample_data](sample_data)のtokyo2015.csvをコピーしてください。

Logstashを使ってサンプルデータをElasticsearchに登録します。次のセルのコマンドを実行してください。

In [72]:
!ansible logstash-server -i /etc/ansible/hosts -m shell -a "cat tokyo2015.csv | sudo /usr/share/logstash/bin/logstash --path.settings /etc/logstash -f /etc/logstash/conf.d/main.conf" -u $USER --private-key=$KEYPATH

XXX.XXX.XXX.232 | SUCCESS | rc=0 >>
Sending Logstash logs to /var/log/logstash which is now configured via log4j2.properties.



データが登録されているかどうかは、次のセルを実行することで確認できます。

In [73]:
!curl -XGET http://$ES_CLIENT/$INDEX/_search/?pretty

{
  "took" : 13,
  "timed_out" : false,
  "_shards" : {
    "total" : 70,
    "successful" : 70,
    "failed" : 0
  },
  "hits" : {
    "total" : 358,
    "max_score" : 1.0,
    "hits" : [
      {
        "_index" : "meteorological-data-2015.01",
        "_type" : "logs",
        "_id" : "tokyo_15-01-08",
        "_score" : 1.0,
        "_source" : {
          "date" : "15-01-08",
          "wind_speed_avg" : "4.4",
          "snowfall" : null,
          "temperature_max" : "11.2",
          "precipitation_day" : null,
          "wind_speed_max" : "8.4",
          "sunshine_duration" : "8.9",
          "temperature_avg" : "6.2",
          "temperature_min" : "2.0",
          "@version" : "1",
          "wind_direction_max_moment" : "北西",
          "information_night" : "晴",
          "precipitation_max_10min" : null,
          "precipitation_max_hour" : null,
          "wind_direction" : "西",
          "information_daytime" : "晴",
          "@timestamp" : "2015-01-07T15:00:00.000Z",
  

登録したデータの内容がJSON形式で表示されるので、それを確認してください。  
レスポンスのサイズを指定していないため、ここでは最大10件まで確認できます。  
データが登録されていることが確認できたら、セットアップ作業は終了です。

## 構築するにあたって

構築にあたり、1サーバのスペック、設定するshardの数、各役割のノード数を決める必要があります。  
その際の指針となる考えを示します。  
具体的な数値に関しては用途に応じて決定してください。

![images/01_server_construction.png](images/01_server_construction.png)

### 1サーバ当たりのスペック

Elasticsearchに何をさせるかによってボトルネックになる箇所が異なるため、  
ハードウェアのスペックを決定する際に、ある程度「どういう処理をするのか」を想定しておく必要があります。  

Elasticsearchを利用する際に特に検討が必要なスペックは以下の通りです。  

- CPU: 全体の処理速度とコア数はクエリの並列処理  
- ディスク: 検索とindexの頻度(データ登録の頻度)  
- メモリ: キャッシュとして利用されるため、ディスクと同じく検索のレスポンスに影響します。  

ここでは良くある2つのユースケースを示します。

#### ユースケース1  

検索処理を高速に行いたい場合

このケースではクエリを並列に実行させて、検索レスポンスの速度を確保する必要があります。  
クエリはshardごとに並列で実行されるため、CPUコア数までshard数を設定する、サーバ台数を増やす、といった対応を行います。  

また、検索速度を上げるにはディスクの速度だけではなく、OSのディスクキャッシュを潤沢に確保する必要があります。  
単純な検索クエリについてElastic社が性能検証を行った結果、もっともパフォーマンスが出やすいのが「メモリ:ディスク=1:16」と実測されました。  
例えば、メモリが64GBの場合には1TBのデータ保持が適切です。  

※メモリとディスク容量の比率はElastic社の資料[Sizing Scenario](https://speakerdeck.com/elastic/quantitative-cluster-sizing?slide=16)を参考にしています。

|メモリ(GB)|ディスク(GB)|
|---------|-----------|
| 32      | 512       |
| 64      | 1024      |
<center>メモリとディスクの割合を1:16で計算した場合</center>

#### ユースケース2 

色々なデータを大量に持たせたい場合  

ユースケース1に記載した以上にディスクの比率を大きくし、1ノードのデータ量を増やした場合、ディスクキャッシュのヒット率が下がり、ディスクアクセスが増加するため、検索速度が遅くなります。  
どの程度までの検索時間を許容するかに依存しますが、1ノードのデータ量はユースケース1の10倍程度までが目安となります。

### Shard数を決める

Elasticsearchの使用用途によって、設定するshard数も変わってきます。  
1.6.1で上げた2つのユースケースの例で考えると、

ユースケース1では、サーバ1台のCPUコア数と同じ数を設定します。  
ユースケース2では、shard数を3～5程度にし、サーバ1台のデータ容量を多く持てるようにすると良いです。

shard数を多くするメリット・デメリットは以下が考えられます。    
- メリット: クエリの並列数が上がり、1ノード当たりのパフォーマンス向上が見込める  
- デメリット: shard数をCPUコア数より大きくしても並列度が上がらず、検索速度は向上しない 

設定方法としてshard数を縮小するためのAPIが用意されています。  
詳細はElasticsearch Referenceの[Shrink Index](https://www.elastic.co/guide/en/elasticsearch/reference/current/indices-shrink-index.html)を参照してください。

また、shard数はクエリ解析時にも影響があります。  
詳細は[07_Troubleshooting](07_Troubleshooting.ipynb#Slowログを確認)を参照してください。

### サーバ台数の決め方

もとめられる検索速度、扱うデータ量、用意できるサーバスペックによってクラスタを構成する台数は大きく変化します。  
そのため、正確な見積もりは難しいため、ベンチマークを実施することを推奨します。  
ここでは、データ、マスターノード数を決定する上での考えを示します。

#### データノード数

以下にデータノード数を決める流れを示します。  

**①データサイズ(保持するデータ量) を決定する**  
データの内容により、データサイズは異なります。  
必要なデータ量の何分の1かをElasticsearchに登録し、そのとき利用したディスク使用量から、全体のデータ量を算出してください。

**②必要メモリサイズ(①のデータサイズを扱うために必要なメモリ量)を決定する**  
次に、メモリとディスクの比率から、必要なメモリサイズを算出します。  
ここでは、ユースケース1を想定し、メモリ:ディスクの比率として1:16とし、1ノード当たりメモリ64GB、ディスク1TBとします。  

*必要メモリサイズ = ディスク / 16 *

ノード1台あたりのメモリサイズの詳細はElasticsearch Referenceの[Memory](https://www.elastic.co/guide/en/elasticsearch/guide/current/hardware.html#_memory)を参照してください。

**③データノード数を決定する**  
必要なデータ量から、それを満たすデータノード数を算出します。  
このケースでは、ノード1台あたりのディスクサイズは1TBとしているため、こちらを使用して計算します。  

*データノード数 = 全体のデータ量 / 1(TB)　※ただし、冗長性を考慮し、最低3台とする。*

#### マスターノード数

マスターノード数は、Split brain問題を避けるために3台とすることが望ましいです。  
Split brainとは、クラスタ内でMaster nodeが複数存在してしまいデータが失われる危険性がある問題です。  
詳細はElasticsearch Referenceの[Master Eligible Node](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/modules-node.html#split-brain)を参照してください。

### Elasticsearch本体のパラメータチューニング

パフォーマンスを改善するための主なパラメータは次のようなものになります。  
設定にはindexごとの設定や、ノードごとの設定、クラスタ全体の設定があり、設定方法が異なります。  
設定に関しての詳細はElasticsearch Referenceの[Configuring Elasticsearch](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/settings.html)を参照してください。

| チューニング項目         | 設定項目名            | チューニング概要       |詳細の参照先       |設定方法                                                                                                                                                                                                      |
|--------------------------|-----------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|---------------------------------------|
| [シャード数](#Shard数を決める)               | number_of_shards      | シャード数を増やすことで書き込みを分散します。|[Index Modules](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/index-modules.html#dynamic-index-settings)       |indexごとに設定することができます。REST APIから設定します。                                                                                                                                                                                      |
| [indexの更新間隔](#1サーバ当たりのスペック)          | refresh_interval      | indexの更新間隔を広げることで、インデックス処理は遅延しますが、Elasticsearchの負荷を下げることができます。|[Index Modules](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/index-modules.html#dynamic-index-settings)         |indexごとに設定することができます。REST APIから設定します。                                                                                                                                            |
| [リクエストキューのサイズ](#サーバ台数の決め方) | thread_pool.queue.size | リクエストキューのサイズを増やすことで、高負荷時のエラー発生を防ぐことができます。<br>キューが溢れた場合、Elasticsearchはドキュメントを破棄し、429エラーを返します。<br>Logstashを用いてドキュメント登録を行っている場合、下記のようなエラーが出力されます。<br>その場合はキューのサイズを増やすか、Elasticsearchの処理速度を向上させる必要があります。<br><br>17:42:11.840 [[main]>worker6] INFO  logstash.outputs.elasticsearch - retrying failed action with response code: 429 ({"type"=>"es_rejected_execution_exception", "reason"=>"rejected execution of org.elasticsearch.transport.TransportService$6@1adec92 on EsThreadPoolExecutor[bulk, queue capacity = 0, org.elasticsearch.common.util.concurrent.EsThreadPoolExecutor@75905a2[Running, pool size = 1, active threads = 0, queued tasks = 0, completed tasks = 1]]"})|[Thread Pool](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/modules-threadpool.html)             |elasticsearch.ymlで設定することができます。                                                                                                                                                        |
| [バルク処理のスレッド数](#1サーバ当たりのスペック)   | thread_pool.bulk.size  | バルクインポート処理（複数のドキュメントをまとめて追加する処理）を受け付けるスレッド数を増減させます。<br>スレッド数を増加させることによって処理が並列化され、パフォーマンスの向上が期待されます。<br>ただし、稼働マシンのコア数を超える値を設定できないように制限されています。|[Thread Pool](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/modules-threadpool.html)   |elasticsearch.ymlで設定することができます。                                                                                              

## Elasticsearchが得意としないクエリ

### 時系列データの連続する期間

Elasticsearchは「保有データの中から該当するデータを見つける、見つけたデータを集計する」点に強みがありますが、不得意とする集計方法も存在します。  
「連続して条件を満たすデータの範囲を求める」「条件が変化する箇所を見つける」ようなこと不得意な処理です。 

例えば、「ある期間の中で正→負→正と値が切り替わる回数を集計する」といった処理です。  
こういった解析は[03_Aggregation](03_Aggregation.ipynb)のBucket SelectorやSerial Differencing機能を組み合わせることで実現可能ですが、複雑なクエリとなり、実行時間も長くなるため、注意が必要です。  

### 正規化されたデータ

リレーショナルデータベースのようなデータのJOINをできないため、データの投入時に正規化を崩す必要があります。  
例として１つの計測データと、それが参照するマスターデータがある場合を考えましょう。

このようなデータの場合、以下のようなドキュメントを登録します。

正規化しないことでデータ量は増えますが、Elasticsearchではこのようにデータを持つ必要があります。

### 同時更新が必要なもの

トランザクション機能がないため、複数クライアントから同じドキュメントを同時に更新すると、期待と異なる動作をする可能性があります。  
この問題を回避する方法については、Elasticsearch Referenceの[Solving Concurrency Issues](https://www.elastic.co/guide/en/elasticsearch/guide/2.x/concurrency-solutions.html)を参照ください。